### Audio Transcriber using OPENAI WHISPER
Live rolling window transcript using chunkung of audio file
Complete transcript using full audio file after CTRL+C

In [1]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np
import whisper
import sounddevice as sd
import numpy as np
from collections import deque
import tempfile
from scipy.io.wavfile import write
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading

"""
pip install openai-whisper
pip install sounddevice
pip install scipy
pip install numpy
pip install torch
pip install ipywidgets
"""

'\npip install openai-whisper\npip install sounddevice\npip install scipy\npip install numpy\npip install torch\npip install ipywidgets\n'

In [2]:
# ----- PARAMETERS -----
samplerate = 16000
channels = 1
chunk_sec = 1      # 1 second per chunk
window_sec = 5     # rolling window for live preview
final_audio_path = "final_audio.wav"
final_transcript_path = "final_transcript.txt"

save_path = "C:\\Users\\sampw\\OneDrive\\Code Projects\\Python\\Projects\\Multi Tool\\AudioTranscription\\out"
final_audio_path = os.path.join(save_path, "final-audio.wav")
final_transcript_path = os.path.join(save_path, "final-transcript.txt")

In [3]:
print("Loading Whisper model (medium)...")
model = whisper.load_model("medium")
print("Model loaded!")

Loading Whisper model (medium)...
Model loaded!


In [ ]:
# ----- STATE -----
rolling = deque(maxlen=samplerate * window_sec)
full_audio = []
stop_flag = False

# ----- WIDGETS -----
message = widgets.Label(value="🎤 Recording... press Stop when done.")
stop_button = widgets.Button(description="Stop Recording", button_style="danger")
display(message, stop_button)

def stop_recording(b):
    global stop_flag
    stop_flag = True
    message.value = "🛑 Stopping... please wait."

stop_button.on_click(stop_recording)

# ----- HELPER -----
def timestamp():
    return datetime.now().strftime("%H:%M:%S")

# ----- RECORDING LOOP -----
def recording_loop():
    global stop_flag
    while not stop_flag:
        chunk = sd.rec(int(chunk_sec * samplerate),
                       samplerate=samplerate,
                       channels=channels)
        sd.wait()
        
        full_audio.append(chunk.copy())
        rolling.extend(chunk.flatten())
        
        # ----- LIVE PREVIEW -----
        tmp = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
        window_audio = np.array(rolling, dtype=np.float32)
        write(tmp.name, samplerate, window_audio)

        # preview = model.transcribe(tmp.name, task="translate")  # replace as needed
        tmp.close()
        os.remove(tmp.name)

        # Update GUI and print timestamped line
        text = preview['text'].strip()
        if text:
            print(f"[{timestamp()}] {text}")
        message.value = f"🌐 Live: {text}"

    # # ----- SAVE FINAL AUDIO & TRANSCRIPT -----
    # full_array = np.concatenate(full_audio).astype(np.float32)
    # write(final_audio_path, samplerate, full_array)
    # print(f"[{timestamp()}] 💾 Saved full audio: {final_audio_path}")
    # message.value = f"💾 Saved full audio: {final_audio_path}"

    # message.value = "📝 Transcribing full audio (this may take a moment)…"
    # result = model.transcribe(final_audio_path, task="translate")
    # with open(final_transcript_path, "w", encoding="utf-8") as f:
    #     f.write(result["text"])
    # print(f"[{timestamp()}] ✨ Final transcript saved: {final_transcript_path}")
    # message.value = f"✨ Final transcript saved: {final_transcript_path}"

# Run recording in background thread
thread = threading.Thread(target=recording_loop, daemon=True)
thread.start()

Label(value='🎤 Recording... press Stop when done.')

Button(button_style='danger', description='Stop Recording', style=ButtonStyle())

C:\Users\sampw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\whisper\transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Exception in thread Thread-55:
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 980, in _bootstrap_inner
    self.run()
  File "C:\Users\sampw\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\sampw\AppData\Local\Temp\ipykernel_32136\4268